# MBA DSA USP/ESALQ 2024.1 Time 2
## Desafio House Prices

## A fazer:
- Limpeza geral
- Análise das variáveis (categóricas/numéricas(discretas-contínuas)/trocadas)
- Outliers?
- Normalização dos dados de regressão não-gaussianos
- Análise de correlação
- Escolha das variáveis categóricas/numéricas
- Imputação dos NaN
- Multicolineariedade?

## Carregamento dos dados

Importação das bibliotecas necessárias

In [305]:
import pandas as pd

Leitura dos dados amplos (arquivos de treino/teste) para unificação e tratamento por completo antes da modelagem.

In [306]:
treino = pd.read_csv("train.csv")
teste = pd.read_csv("test.csv")

# Ajuste das dimensões - inclusão da coluna SalePrice no dataset de teste
teste["SalePrice"] = 0

# Inclusão de informação sobre dataset de origem
teste["Origem"] = "Teste"
treino["Origem"] = "Treino"

dados_brutos = pd.concat([treino, teste])

print("Quantidade de linhas e colunas")
print(dados_brutos.shape)

Quantidade de linhas e colunas
(2919, 82)


## Análise das variáveis categóricas/de classificação

O banco de dados possui uma série de colunas (variáveis) que possuem valores nulos, algumas em maior quantidade e outras em menor quantidade, que serão revisitados depois:

Separação das variáveis entre categóricas e numéricas.

In [307]:
coluna_origem = dados_brutos["Origem"]

colunas_quali = dados_brutos.columns[dados_brutos.dtypes == "object"]
variaveis_quanti = dados_brutos.drop(colunas_quali, axis = 1)

colunas_quanti = dados_brutos.columns[dados_brutos.dtypes != "object"]
variaveis_quali = dados_brutos.drop(colunas_quanti, axis = 1)

"""
#print("AS VARIÁVEIS NUMÉRICAS ('QUANTI') ENTÃO SÃO " + str(len(variaveis_quanti.columns)) + ":")
#print(variaveis_quanti.columns)
#print("\n")

#print("AS VARIÁVEIS NÃO-NÚMERICAS ('QUALI') FICAM SENDO APENAS " + str(len(variaveis_quali.columns)) + ":")
#print(variaveis_quali.columns)
#print("\n")

"""
#print("REINSERÇÃO DA COLUNA 'ORIGEM' PARA CONTROLE NAS VARIÁVEIS QUANTI:")
variaveis_quanti["Origem"] = coluna_origem

### Variáveis de Classificação

Algumas variáveis tem valores nulos, que serão imputados com classificações apropriadas. O processo de decisão sobre a imputação é tratado caso a caso.

In [308]:
nulos = variaveis_quali.isnull().sum()
nulos_ordenados = (nulos.sort_values(ascending=False)/len(variaveis_quali))*100

"""
print("Percentual de valores nulos por cada variável:")
print(nulos_ordenados.head(25))
print("\nComo os últimos valores já são zero, depois desses todos são zerados mesmo. Ou seja, não existem valores nulos nas outras variáveis")
"""

'\nprint("Percentual de valores nulos por cada variável:")\nprint(nulos_ordenados.head(25))\nprint("\nComo os últimos valores já são zero, depois desses todos são zerados mesmo. Ou seja, não existem valores nulos nas outras variáveis")\n'

Verificação das variáveis com maior falta de dados. Como tratam-se de variáveis de classificação, muitas vezes a imputação é simples e válida de ser feita.

In [ ]:
"""
print("Variáveis com % significativo de falta de dados (acima de 20%):")
print(nulos_ordenados.head(5))
print("\nAs demais variáveis terão todos seus valores NaN imputados ou as observações NaN serão desconsideradas.")
"""

Descrição das variáveis com maior falta de dados: 
- PoolQC: valores NaN indicam inexistência de piscina (imputar 'None' então)
- MiscFeature: miscelânea, NaN indica inexistência desse tipo de feature (imputar 'None')
- Alley: tipo de viela/ruazinha para acesso à casa. Se a casa tem saída para a rua principal, a característica não tem nem como existir (imputar 'None')
- Fence: qualidade da cerca da casa, se a casa não possui cerca por obviedade essa informação é NaN (imputar 'None')
- FireplaceQu: indica existência de lareira na casa (imputar 'None')

In [309]:
imputar_none = ["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"]
variaveis_quali[imputar_none] = variaveis_quali[imputar_none].fillna("None")


#### Demais variáveis

In [366]:
nulos = variaveis_quali.isnull().sum()
nulos_ordenados = nulos.sort_values(ascending=False)

temp = pd.DataFrame(nulos_ordenados)
mask = temp[0].isin([0])
print(temp[~mask].index)

Index([], dtype='object')


Imputação de 'None' diretamente:
- "Garage"XYZ: as variáveis "Garage"XYZ classificam o estado de conservação, acabamento e localização da garagem, se existente. NaN indica inexistência de garagem (imputa 'None').
- "Bsmt"XYZ: as variáveis "Bsmt"XYZ classificam o nível de qualidade, exposição e acabamentos do porão, se existente. NaN indica inexistência de porão (imputa 'None').
- Exterior1st: Exterior covering on house. Deduzo que NaN indique inexistência, imputa 'None'.
- Exterior2nd: Exterior covering on house (if more than one material) (deve ter correlação direta com o acima).
- MasVnrType: Masonry veneer type. Descrição contempla opção de 'None', assume-se que informação inexistente corresponda a 'None'.

In [342]:
imputar_none2 = ['GarageCond', 'GarageQual', 'GarageFinish', 'GarageType', 'BsmtCond', 'BsmtExposure', 'BsmtQual', 'BsmtFinType2', 'BsmtFinType1','Exterior1st', 'Exterior2nd', 'MasVnrType']
variaveis_quali[imputar_none2] = variaveis_quali[imputar_none2].fillna("None")

Imputação arbitrada para a classificação:
- Functional: Home functionality (Assume typical unless deductions are warranted). NaN imputado "Typ".
- SaleType: tipo de venda, imputado moda de "OverallCond" (banco de dados original).
- Utilities: tipos de serviços básicos disponíveis (água, luz, gás, etc.). Imputada moda da vizinhança.
- KitchenQual: qualidade da cozinha. Imputada moda da vizinhança.
- Electrical: Electrical system. Imputada moda da vizinhança.
- MSZoning: Identifies the general zoning classification of the sale. Como se trata de zoneamento, assume-se que em um mesmo bairro o zoneamento seja o mesmo(sem NaN na descrição).

In [ ]:
# Variaveis imputadas com moda da região (vizinhança)

variaveis_quali["Functional"] = variaveis_quali["Functional"].fillna("Typ")
variaveis_quali["Utilities"] = variaveis_quali["Utilities"].fillna("AllPub")
variaveis_quali["SaleType"] = variaveis_quali["SaleType"].fillna("WD")
variaveis_quali["MSZoning"].loc[1444] = variaveis_quali["MSZoning"].loc[1444].fillna("RL") # específico para essa observação o bairro é diferente, com outra moda
variaveis_quali["MSZoning"] = variaveis_quali["MSZoning"].fillna("RM")


# Processo da imputação ("KitchenQual", nessa explicação)
# Primeiro vê qual observação é NaN, extraindo a informação sobre vizinhança
kitchen_neigh = variaveis_quali[variaveis_quali["KitchenQual"].isnull()][["Neighborhood"]]
# Com a informação sobre a vizinhança, verifica a moda da vizinhança para a variável com NaN
kitchen_qual = variaveis_quali[variaveis_quali["Neighborhood"] == kitchen_neigh.iloc[0,0]][["KitchenQual"]].mode()
# Por último, imputa a moda da vizinhança para a informação faltante
variaveis_quali["KitchenQual"] = variaveis_quali["KitchenQual"].fillna(kitchen_qual.iloc[0,0])


# Repete o processo acima, para a variável "Electrical"
elec_neigh = variaveis_quali[variaveis_quali["Electrical"].isnull()][["Neighborhood"]]
elec_qual = variaveis_quali[variaveis_quali["Neighborhood"] == elec_neigh.iloc[0,0]][["Electrical"]].mode()
variaveis_quali["Electrical"] = variaveis_quali["Electrical"].fillna(elec_qual.iloc[0,0])

### Variáveis Numéricas

In [373]:

nulos_quanti = variaveis_quanti.isnull().sum()
nulos_ordenados_quanti = (nulos_quanti.sort_values(ascending=False)/len(variaveis_quanti))*100


print("Percentual de valores nulos por cada variável:")
print(nulos_ordenados_quanti.head(12))
print("\nComo os últimos valores já são zero, depois desses todos são zerados mesmo. Ou seja, não existem valores nulos nas outras variáveis")


Percentual de valores nulos por cada variável:
LotFrontage     16.649538
GarageYrBlt      5.447071
MasVnrArea       0.787941
BsmtHalfBath     0.068517
BsmtFullBath     0.068517
BsmtFinSF2       0.034258
GarageCars       0.034258
GarageArea       0.034258
TotalBsmtSF      0.034258
BsmtUnfSF        0.034258
BsmtFinSF1       0.034258
Fireplaces       0.000000
dtype: float64

Como os últimos valores já são zero, depois desses todos são zerados mesmo. Ou seja, não existem valores nulos nas outras variáveis


As variáveis com valores nulos:

- LotFrontage - Linear feet of street connected to property
- GarageYrBlt - Ano em que a garagem foi construída. Se existe um NaN, assume-se que não tenha garagem, atribuindo-se 'None'. Transformar essa variável para classificação, uma vez que o ano de construção não é valor métrico. 
- MasVnrArea - Masonry veneer area in square feet. Deve ter relação direta com a MasVnrType - se esta é inexistente, então a área é 0 mesmo.
- BsmtHalfBath - 'meio' banheiro no porão, se inexistente imputa 0. Classificação?
- BsmtFullBath - banheiro no porão, se inexistente imputa 0. Classificação?
- BsmtFinSF2 - área do banheiro no porão, se inexistente imputa 0.
- GarageCars - tamanho da garagem em questão de capacidade de números de carros, se garagem inexistente então 0.
- GarageArea - tamanho da garagem, se garagem inexistente então 0.
- TotalBsmtSF - tamanho total do porão, se porão inexistente então 0.
- BsmtUnfSF - área inacabada do porão, se porão inexistente então 0.
- BsmtFinSF1 - área acabada do porão, se porão inexistente então 0.


Com o ensejo da variável GarageYrBlt, transformar as variáveis de 'ano' em classificação. Algumas variáveis de ano (como ano de construção, reforma ou venda) serão utilizadas apenas para engenharia de features, neste caso então não interfere se o ano enquandra-se como variável métrica ou de classificação.

Somar áreas de banheiro?
Somar áreas totais?
Somar quantidade total de banheiros?

As variáveis "xyzBath" são de classificação?

Como usa as variáveis de classificação e numéricas juntas? Passa as de classificação tudo para dummies e faz regressão normal?

# O TRABALHO INICIAL É DAQUI PARA BAIXO

 Filtro das variáveis quali e quanti para evitar confusão. Contudo, ainda existem valores nulos no meio dos dados (foram eliminados só as variações com nulos >= 15% do total). Necessário realizar a imputação esses valores, pois nenhuma análise é feita com qualquer valor nulo: quebra o sistema.

 Link sobre o carregamento das bibliotecas para imputar dados: https://medium.com/@sanjushusanth/missing-value-imputation-techniques-in-python-62aeab65a6a6

 Por mera facilidade momentânea, serão utilizados os dados de média e mediana para as imputações, ou, ainda seguindo o que alguns consideram 'boas práticas', simplesmente registrando um '-1' no valor imputado. Fiz um banco de dados para cada tipo dessas imputações a fim de testar qual melhor se encaixa nessa simples imputação (sem cálculos próprios de imputação para esse fim).

In [6]:
variaveis_quanti_namum = variaveis_quanti.fillna(-1)
variaveis_quanti_namediana = variaveis_quanti.fillna(variaveis_quanti.median())
variaveis_quanti_namedia = variaveis_quanti.fillna(variaveis_quanti.mean())

print("VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO '-1' AINDA COM VALORES NULOS...")
print(variaveis_quanti_namum.columns[variaveis_quanti_namum.isnull().sum() > 0])

print("\n VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO 'MÉDIA' AINDA COM VALORES NULOS...")
print(variaveis_quanti_namum.columns[variaveis_quanti_namedia.isnull().sum() > 0])

print("\n VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO 'MEDIANA' AINDA COM VALORES NULOS...")
print(variaveis_quanti_namum.columns[variaveis_quanti_namediana.isnull().sum() > 0])

try:
    variaveis_quanti_namum.columns[variaveis_quanti_namediana.isnull().sum() > 0] + variaveis_quanti_namum.columns[variaveis_quanti_namedia.isnull().sum() > 0]+ variaveis_quanti_namum.columns[variaveis_quanti_namum.isnull().sum() > 0]
    print("\nTUDO CERTO!")
except:
    print("\n!!! DEU ALGUM ERRO !!!")

VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO '-1' AINDA COM VALORES NULOS...
Index([], dtype='object')

 VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO 'MÉDIA' AINDA COM VALORES NULOS...
Index([], dtype='object')

 VERIFICANDO VARIÁVEIS C/ IMPUTAÇÃO 'MEDIANA' AINDA COM VALORES NULOS...
Index([], dtype='object')

TUDO CERTO!


Definição da variável dependente (y) e das variáveis independentes (x) de cada um dos bancos de dados anteriores:

In [7]:
x_namum = variaveis_quanti_namum.drop("SalePrice", axis = 1)
x_namedia = variaveis_quanti_namedia.drop("SalePrice", axis = 1)
x_namediana = variaveis_quanti_namediana.drop("SalePrice", axis = 1)
y = variaveis_quanti.SalePrice # como o 'y' é igual para todos (o preço de venda), então tanto faz a base de dados escolhida

### Separação dos dados para treino e teste

Importação das bibliotecas adicionais necessárias

In [8]:
from sklearn.model_selection import train_test_split

Separação do banco de dados entre treino (0,80 ou 80%) e teste (0,20 ou 20%) - proporções arbitrárias, eu que defini assim. Pode-se testar com proporções diferentes. Usa-se o scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Para garantir consistência entre os resultados, utilizado sempre _random_state = 1_

In [9]:
# Separação treino e teste dos dados imputados com -1
treino_namum, teste_namum, treino_y_um, teste_y_um = train_test_split(x_namum, y, test_size = 0.2, random_state = 1)

# Separação treino e teste dos dados imputados com a média
treino_namedia, teste_namedia, treino_y_media, teste_y_media = train_test_split(x_namedia, y, test_size = 0.2, random_state = 1)

# Separação treino e teste dos dados imputados com a mediana
treino_namediana, teste_namediana, treino_y_mediana, teste_y_mediana = train_test_split(x_namediana, y, test_size = 0.2, random_state = 1)

### Modelagem e posterior avaliação dos resultados
Com os dados já limpos, já separados pelas imputações diferentes para verificação e com as bases para treino e teste, começa-se efetivamente a modelagem através de alguns algoritmos diferentes a fim de testar precisão dos resultados estimados.

Importação das bibliotecas adicionais necessárias

In [26]:
from sklearn import tree # biblioteca para modelagem através de decision tree (árvore de decisão)
from sklearn.neighbors import KNeighborsRegressor # biblioteca para modelagem através de KNN
from sklearn.ensemble import GradientBoostingRegressor # biblioteca para modelagem com Gradient Boost
from sklearn.linear_model import LinearRegression # biblioteca para modelagem através de regressão Linear Simples
from sklearn.linear_model import LogisticRegression # biblioteca para modelagem com Logistic Regression
from sklearn.linear_model import Ridge # biblioteca para modelagem com Ridge
from sklearn import linear_model # biblioteca para modelagem com Bayesian Ridge

#### Documentação
Linear Regression (Regressão Linear)
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression

Decision Tree (Árvore de Decisão)
https://scikit-learn.org/stable/modules/tree.html#regression

KNN - K Nearest Neighbors (Vizinhança Próxima)
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html

Gradient Boost Regressor
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor

Logistic Regression (Modelo Logístico, 'Logit' ou Classificador de Máxima Entropia)
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

Ridge
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge

Bayesian Ridge Regression
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html#sklearn.linear_model.BayesianRidge

#### NaN imputado com -1

In [32]:
# Regressão Linear Simples (Linear Regression)
namum_rl = LinearRegression().fit(treino_namum, treino_y_um)
y_namum_rl = namum_rl.predict(teste_namum)

# Árvore de Decisão (decision tree)
namum_tree = tree.DecisionTreeRegressor(random_state = 1).fit(treino_namum, treino_y_um)
y_namum_tree = namum_tree.predict(teste_namum)

#(K Vizinhos Mais Próximos - KNN (K Nearest Neighbors))
namum_knn = KNeighborsRegressor(n_neighbors = 2, weights = "uniform").fit(treino_namum, treino_y_um)
y_namum_knn = namum_knn.predict(teste_namum)

#Gradient Boost Regressor
namum_gbr = GradientBoostingRegressor(random_state = 1).fit(treino_namum, treino_y_um)
y_namum_gbr = namum_gbr.predict(teste_namum)

#Logistic Regression
namum_lor = LogisticRegression(random_state = 1, max_iter = 1000).fit(treino_namum, treino_y_um)
y_namum_lor = namum_lor.predict(teste_namum)

#Ridge
namum_ridge = Ridge(alpha = 1.0).fit(treino_namum, treino_y_um)
y_namum_ridge = namum_ridge.predict(teste_namum)

#Bayesian Ridge Regression
namum_brr = linear_model.BayesianRidge().fit(treino_namum, treino_y_um)
y_namum_brr = namum_brr.predict(teste_namum)

/usr/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### NaN imputado com Média

In [33]:
# Regressão Linear Simples (Linear Regression)
namedia_rl = LinearRegression().fit(treino_namedia, treino_y_media)
y_namedia_rl = namedia_rl.predict(teste_namedia)

# Árvore de Decisão (decision tree)
namedia_tree = tree.DecisionTreeRegressor(random_state = 1).fit(treino_namedia, treino_y_media)
y_namedia_tree = namedia_tree.predict(teste_namedia)

#(K Vizinhos Mais Próximos - KNN (K Nearest Neighbors))
namedia_knn = KNeighborsRegressor(n_neighbors = 2, weights = "uniform").fit(treino_namedia, treino_y_media)
y_namedia_knn = namedia_knn.predict(teste_namedia)

#Gradient Boost Regressor
namedia_gbr = GradientBoostingRegressor(random_state = 1).fit(treino_namedia, treino_y_media)
y_namedia_gbr = namedia_gbr.predict(teste_namedia)

#Logistic Regression
namedia_lor = LogisticRegression(random_state = 1, max_iter = 1000).fit(treino_namedia, treino_y_media)
y_namedia_lor = namedia_lor.predict(teste_namedia)

#Ridge
namedia_ridge = Ridge(alpha = 1.0).fit(treino_namedia, treino_y_media)
y_namedia_ridge = namedia_ridge.predict(teste_namedia)

#Bayesian Ridge Regression
namedia_brr = linear_model.BayesianRidge().fit(treino_namedia, treino_y_media)
y_namedia_brr = namedia_brr.predict(teste_namedia)

/usr/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### NaN imputado com Mediana

In [34]:
# Regressão Linear Simples (Linear Regression)
namediana_rl = LinearRegression().fit(treino_namediana, treino_y_mediana)
y_namediana_rl = namediana_rl.predict(teste_namediana)

# Árvore de Decisão (decision tree)
namediana_tree = tree.DecisionTreeRegressor(random_state = 1).fit(treino_namediana, treino_y_mediana)
y_namediana_tree = namediana_tree.predict(teste_namediana)

#(K Vizinhos Mais Próximos - KNN (K Nearest Neighbors))
namediana_knn = KNeighborsRegressor(n_neighbors = 2, weights = "uniform").fit(treino_namediana, treino_y_mediana)
y_namediana_knn = namediana_knn.predict(teste_namediana)

#Gradient Boost Regressor
namediana_gbr = GradientBoostingRegressor(random_state = 1).fit(treino_namediana, treino_y_mediana)
y_namediana_gbr = namediana_gbr.predict(teste_namediana)

#Logistic Regression
namediana_lor = LogisticRegression(random_state = 1, max_iter = 1000).fit(treino_namediana, treino_y_mediana)
y_namediana_lor = namediana_lor.predict(teste_namediana)

#Ridge
namediana_ridge = Ridge(alpha = 1.0).fit(treino_namediana, treino_y_mediana)
y_namediana_ridge = namediana_ridge.predict(teste_namediana)

#Bayesian Ridge Regression
namediana_brr = linear_model.BayesianRidge().fit(treino_namediana, treino_y_mediana)
y_namediana_brr = namediana_brr.predict(teste_namediana)

/usr/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Avaliação dos resultados realizada através de RMSE (root mean squared error), no caso deste desafio.

Importação das bibliotecas adicionais necessárias

In [35]:
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error
import math

#### Análises (pela imputação)

##### NaN por -1

In [36]:
print("Linear Regression")
print(root_mean_squared_error(teste_y_um, y_namum_rl))
print("\nDecision Tree")
print(root_mean_squared_error(teste_y_um, y_namum_tree))
print("\nKNN")
print(root_mean_squared_error(teste_y_um, y_namum_knn))
print("\nGBR")
print(root_mean_squared_error(teste_y_um, y_namum_gbr))
print("\nLogistic Regression")
print(root_mean_squared_error(teste_y_um, y_namum_lor))
print("\nRidge")
print(root_mean_squared_error(teste_y_um, y_namum_ridge))
print("\nBayesian Ridge")
print(root_mean_squared_error(teste_y_um, y_namum_brr))

Linear Regression
35621.27326115689

Decision Tree
35116.94890306209

KNN
53538.68340927549

GBR
22404.11312398332

Logistic Regression
56992.054928675374

Ridge
35608.081379361436

Bayesian Ridge
40229.11392691891


##### NaN por Média

In [37]:
print("Linear Regression")
print(root_mean_squared_error(teste_y_media, y_namedia_rl))
print("\nDecision Tree")
print(root_mean_squared_error(teste_y_media, y_namedia_tree))
print("\nKNN")
print(root_mean_squared_error(teste_y_media, y_namedia_knn))
print("\nGBR")
print(root_mean_squared_error(teste_y_media, y_namedia_gbr))
print("\nLogistic Regression")
print(root_mean_squared_error(teste_y_media, y_namedia_lor))
print("\nRidge")
print(root_mean_squared_error(teste_y_media, y_namedia_ridge))
print("\nBayesian Ridge")
print(root_mean_squared_error(teste_y_media, y_namedia_brr))

Linear Regression
36078.05339518298

Decision Tree
47410.39995583883

KNN
53401.849345830655

GBR
23146.583584564967

Logistic Regression
53848.77888867462

Ridge
36065.19504911939

Bayesian Ridge
40239.30702099713


##### NaN por Mediana

In [39]:
print("Linear Regression")
print(root_mean_squared_error(teste_y_mediana, y_namediana_rl))
print("\nDecision Tree")
print(root_mean_squared_error(teste_y_mediana, y_namediana_tree))
print("\nKNN")
print(root_mean_squared_error(teste_y_mediana, y_namediana_knn))
print("\nGBR")
print(root_mean_squared_error(teste_y_mediana, y_namediana_gbr))
print("\nLogistic Regression")
print(root_mean_squared_error(teste_y_mediana, y_namediana_lor))
print("\nRidge")
print(root_mean_squared_error(teste_y_mediana, y_namediana_ridge))
print("\nBayesian Ridge")
print(root_mean_squared_error(teste_y_mediana, y_namediana_brr))

Linear Regression
36055.310690629514

Decision Tree
34013.71426658524

KNN
53488.18385333421

GBR
23125.9818814507

Logistic Regression
51854.51240034369

Ridge
36042.55239994742

Bayesian Ridge
40228.513586626104


### O modelo para publicação

In [47]:
# GBR com mediana (23125.98188)

# Carregamento do dataset
teste = pd.read_csv("test.csv")

# Filtro das colunas utilizadas e imputação dos NaN
colunas_quali_teste = teste.columns[teste.dtypes == "object"]
variaveis_quanti_teste = teste.drop(colunas_quali_teste, axis = 1)
variaveis_quanti_teste = variaveis_quanti_teste.drop("LotFrontage", axis = 1)
variaveis_quanti_teste = variaveis_quanti_teste.fillna(variaveis_quanti_teste.median())

# Gerando as previsões
y_gbr = namediana_gbr.predict(variaveis_quanti_teste)

### Gerar arquivo de submissão
Pontuação 0.14569 (quanto menor, melhor)

In [52]:
teste["SalePrice"] = y_gbr
resultado = teste[["Id", "SalePrice"]]
resultado.to_csv("resultado.csv", index=False)